<a href="https://colab.research.google.com/github/merlinkalokerinos/Studies-OAMK/blob/main/Munich_rental_ML_model_OAMK%20arthur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation

In [ ]:
# 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !google-drive-ocamlfuse -cc

In [ ]:
# !pip install scikit-learn

In [ ]:
# !pip install lightgbm

In [ ]:
import os 
import yaml 
import pandas as pd
import numpy as np 

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import scatter_matrix

# ML imports
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Evaluation
from sklearn.model_selection import train_test_split
from sklearn import*

import tarfile
import urllib.request

import lightgbm as lgb

# Data transformation

In [ ]:
df01 = pd.read_csv('/content/drive/Othercomputers/My computer/Documents/HNU/OAMK/Lectures OAMK/Data Analysis Case Study/final test.csv')
df02=pd.read_csv('/content/drive/Othercomputers/My computer/Documents/HNU/OAMK/Lectures OAMK/Data Analysis Case Study/data per district/Indikatoren München Gesamt.csv')

df01.head(20)

In [ ]:
df02.head(5)

In [ ]:
# convert room column to int 
df01["rooms"] = df01["rooms"].astype(int)

In [ ]:
# add price per square and round to two decimals
# df01["price per square"] = df01["price"]/df01["square metres"]
df01 = df01.round({'price per square': 2})

In [ ]:
# merge apartment data with demographic data
df=pd.merge(df01,df02, left_on=['city district'], right_on= ['city district'])
df.head()

In [ ]:
# lets count data sets by district
df.groupby('district name')['district name'].count().head(25)

Filter

In [ ]:
# calculate missing attributes
num_missing = len(df) - len(df.dropna())
ratio_missing = num_missing / len(df) 

# convert and print ratios
ratio_missing = np.round(ratio_missing * 100, 3) 
print(f"{ratio_missing}% of data ({num_missing} data points) have missing values!")

In [ ]:
df.loc[df.isnull().sum(axis=1) == 1] # show missing values, if any are left

In [ ]:
df.isnull().sum(axis=0) # count missing values per column, if any are left

# Peek into our data

In [ ]:
pd.set_option('display.max_rows', 30)
print(df.groupby('district name')['price per square'].mean().sort_values(ascending=False))

In [ ]:
# lets collect the price per square values
derived_df = df['price per square']
print(derived_df, "\n")

In [ ]:
df = df.dropna() # drop N/A sets, if any (just in case)
df = df.reset_index(drop=True) # re-index our data

In [ ]:
# lets look at the distribution
sns.displot(data=df, x="price per square", kde=True);

In [ ]:
# display option for whole url / reset to default (50 characters)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)

# view by price, descending
df01.sort_values(by='price', ascending=True)

=> seems like those are hotel rooms rentable per month, this will drastically increase our average price per square meter, since those have up to ten times the average price

In [ ]:
# lets get an overview of table values and check the average price per square
# the average rent price in Germany is around 10€ per square meter
# the average rent price in Munich has surpased 20€ per square meter according to engel & völkers
description = df.describe()
description

=> the average price, as expected, is at around 25€ for our data set, which is high compared to the average suggested by engel & völkers

I SUGGEST DELETION OF ALL ENTRIES THAT DEVIATE FROM THE MEAN BY MORE THAN DOUBLE THE AVERAGE PRICE, ALSO SOME VERY EXPENSIVE APPARTMENTS ARE OUTLIERS, AS WELL AS SOME CHEAP ONES, THAT ARE UP FOR "TRADE", WE SET A LOWER AND UPPER BOUNDARY TO INCREASE DATA QUALITY AND RELIABILIY OF OUR MODEL

In [ ]:
# filter for appartments with very high price or price per square metre
filtered_df = df.query('`price per square` >= 50 | price >= 5000 | price <= 400')
print(filtered_df.count())

In [ ]:
# drop the filtered rows, reindex and build new description table for bins
df = df.drop(filtered_df.index.values)
df = df.reset_index(drop=True) # re-index our data
description = df.describe()
description

In [ ]:
avgkaufpreis = df["price per square"].mean()
print(avgkaufpreis)

In [ ]:
sns.displot(data=df, x="price per square", kde=True);

In [ ]:
sns.displot(data=df, x="price", kde=True);

In [ ]:
# display option for whole url / reset to default (50 characters)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_colwidth', 50)

# view by price, descending
df.sort_values(by='price', ascending=True)

# Prepare testing data

In [ ]:
# Define four equal sets

bins = [0] + list(description["price per square"][
    ["25%", "50%", "75%", "max"]
].astype(float))
print(bins) # 

In [ ]:
# Categorize data according to their bin

df["house_cat"] = pd.cut(
    df["price per square"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)



In [ ]:
# lets visualize the bins
sns.countplot(data=df, x="house_cat");

In [ ]:
# compare the numbers
print(df.groupby("house_cat")["house_cat"].count())

In [ ]:
# defining train and test sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=69)
for train_index, test_index in split.split(df, df["house_cat"]):
    df_train = df.loc[train_index]
    df_test = df.loc[test_index]

In [ ]:
# visualize train bins
sns.countplot(data=df_train, x="house_cat");

In [ ]:
# visualize test bins
sns.countplot(data=df_test, x="house_cat");

In [ ]:
# drop the category from the data frame
train = df_train.drop("house_cat", axis=1)
test = df_test.drop("house_cat", axis=1)



# Correlation calculation

In [ ]:
train.corr()["price per square"].abs().sort_values(ascending=False)

In [ ]:
def engineer_features(df):
    df["ratio_rooms"] = df["square metres"] / df["rooms"]
    return df

In [ ]:
train = engineer_features(train)
test = engineer_features(test)

In [ ]:

dfcorr = train.corr()["price per square"].abs().sort_values(ascending=False)
#dfcorr.head(50)
pd.set_option('display.max_rows', 158)
dfcorr.head(15)

# ML-Coding | Linear regression

In [ ]:

df1=df

# drop unnecessary values
df2= df1.drop(['url', 'scraped location','city district','geo location','district name','price'], 1)

df2 = df2.dropna() # delete N/A values, if any are left
# df2 = df2.reset_index(drop=True) # re-index

def get_features_and_targets(df2):
    x = df2.drop(["price per square"], axis=1).values
    y = np.stack(df2["price per square"])
    return x, y

description = df2.describe()

bins = [0] + list(description["price per square"][
    ["25%", "50%", "75%","max"]
].astype(float))

df2["house_cat"] = pd.cut(
    df2["price per square"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=69)
for train_index, test_index in split.split(df2, df2["house_cat"]):
    df2_train = df2.loc[train_index]
    df2_test = df2.loc[test_index]
    
df2_train = df2_train.drop("house_cat", axis=1)
df2_test = df2_test.drop("house_cat", axis=1)

# convert categories to indicator variables
df2_train_ml = pd.get_dummies(df2_train) # One-Hot Encoding
df2_test_ml = pd.get_dummies(df2_test)

x_train, y_train = get_features_and_targets(df2_train_ml)
x_test, y_test = get_features_and_targets(df2_test_ml)

clf = LinearRegression()
clf.fit(x_train, y_train)


In [ ]:
'''
# drop unnecessary values
df2= df1.drop(['url', 'scraped location','city district','geo location','district name','price'], 1)

target = df2["price per square"]
features = df2.drop("price per square",axis=1)

# train and test split
x_train, x_test, y_train, y_test = train_test_split(features,target,test_size=.1,random_state=69)
'''

In [ ]:
'''
# convert categories to indicator variables
x_train = pd.get_dummies(x_train) # One-Hot Encoding
x_test = pd.get_dummies(x_test)

x_train['district code'] = pd.Series(x_train['district code'], dtype='category')
x_test['district code'] = pd.Series(x_test['district code'], dtype='category')
'''

In [ ]:
predictions = clf.predict(x_test)

In [ ]:
print("Prediction\t|True Value")
print("-"*30)
for idx, (pred, annotation) in enumerate(zip(predictions, y_test)):
    if idx == 25:
        break
    pred = round(float(pred),2)
    annotation = float(annotation)
    print(f"{pred}\t\t|"+f"{annotation}")

In [ ]:
mae = metrics.mean_absolute_error(y_test, predictions)
mae_ratio = metrics.mean_absolute_percentage_error(y_test, predictions)
r2_score = metrics.r2_score(y_test, predictions)

In [ ]:
print(f"Average deviation is {np.round(mae)};")
print(f"Failure quota is {np.round(mae_ratio * 100)}%;")
print(f"Coefficient of determination (R^2) is {np.round(r2_score * 100)}%")

In [ ]:
df2_test.iloc[0]

In [ ]:
for x in range(10):
  single_pred = clf.predict([x_test[x]])[0]
  print(f"Predicted price per square is " + str(round(single_pred,2)) + ".")
  print(f"Real value is {y_test[x]}.")
  print(" ")

In [ ]:
print('Training accuracy {:.4f}'.format(clf.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(clf.score(x_test,y_test)))

In [ ]:
# visualizing in a plot
x_ax = range(len(y_test))
plt.figure(figsize=(12, 6))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, predictions, label="predicted")
plt.title("")
plt.xlabel('X')
plt.ylabel('Price')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show() 

# ML Coding | Light GBM

In [ ]:
# display option for whole url / reset to default (50 characters)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_colwidth', 50)

# define testing and target
df2.head(20)
target = df2["price per square"]
features = df2.drop(['price per square','house_cat'],axis=1)

# train and test split
x_train, x_test, y_train, y_test = train_test_split(features,target,test_size=.1,random_state=69)

features.dtypes.head(200)

In [ ]:
# convert categories to indicator variables
x_train = pd.get_dummies(x_train) # One-Hot Encoding
x_test = pd.get_dummies(x_test)

x_train['district code'] = pd.Series(x_train['district code'], dtype='category')
x_test['district code'] = pd.Series(x_test['district code'], dtype='category')


In [ ]:
# fitting model
model = lgb.LGBMRegressor(learning_rate=0.15,max_depth=-5,random_state=69,boosting_type='gbdt',num_leaves=128, objective='regression')
result= model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],verbose=20,eval_metric='logloss')

In [ ]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

In [ ]:
# prediction
y_pred = model.predict(x_test)


In [ ]:
# visualizing in a plot
x_ax = range(len(y_test))
plt.figure(figsize=(12, 6))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_pred, label="predicted")
plt.title("")
plt.xlabel('X')
plt.ylabel('Price')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show() 

In [ ]:
lgb.plot_importance(model,max_num_features=15,height=.5)

In [ ]:
lgb.plot_metric(model)

In [ ]:
lgb.plot_tree(model,figsize=(240,240))

# ML Coding | Light GBM with less indicators


In [ ]:
df3 = df1[['price per square','square metres','rooms', 'district code']]

target = df3["price per square"]
features = df3.drop("price per square",axis=1)

# train and test split
x_train, x_test, y_train, y_test = train_test_split(features,target,test_size=.1,random_state=69)

In [ ]:
# convert categories to indicator variables
x_train = pd.get_dummies(x_train) # One-Hot Encoding
x_test = pd.get_dummies(x_test)

x_train['district code'] = pd.Series(x_train['district code'], dtype='category')
x_test['district code'] = pd.Series(x_test['district code'], dtype='category')

In [ ]:
# fitting model
model = lgb.LGBMRegressor(learning_rate=0.15,max_depth=-5,random_state=69,boosting_type='gbdt',num_leaves=256,objective='regression',task='train')
result= model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],verbose=20,eval_metric='logloss')

In [ ]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

In [ ]:
lgb.plot_importance(model,max_num_features=15,height=.5)

In [ ]:
# prediction
y_pred = model.predict(x_test)

In [ ]:
# visualizing in a plot
x_ax = range(len(y_test))
plt.figure(figsize=(12, 6))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_pred, label="predicted")
plt.title("")
plt.xlabel('X')
plt.ylabel('Price')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show() 

In [ ]:
lgb.plot_metric(model)

In [ ]:
mae = metrics.mean_absolute_error(y_test, y_pred)
mae_ratio = metrics.mean_absolute_percentage_error(y_test, y_pred)
r2_score = metrics.r2_score(y_test, y_pred)

In [ ]:
print(f"Average deviation is {np.round(mae)};")
print(f"Failure quota is {np.round(mae_ratio * 100)}%;")
print(f"Coefficient of determination (R^2) is {np.round(r2_score * 100)}%")